# Make plots

In [1]:

######## snakemake preamble start (automatically inserted, do not edit) ########
import sys; sys.path.extend(['/fh/fast/bloom_j/software/miniconda3/envs/Huanan_market_samples/lib/python3.10/site-packages', '/home/jbloom/.cache/snakemake/snakemake/source-cache/runtime-cache/tmpu709wcxr/file/fh/fast/bloom_j/computational_notebooks/jbloom/2023/Huanan_market_samples/notebooks', '/fh/fast/bloom_j/computational_notebooks/jbloom/2023/Huanan_market_samples/notebooks']); import pickle; snakemake = pickle.loads(b'\x80\x04\x95\xb1\x10\x00\x00\x00\x00\x00\x00\x8c\x10snakemake.script\x94\x8c\tSnakemake\x94\x93\x94)\x81\x94}\x94(\x8c\x05input\x94\x8c\x0csnakemake.io\x94\x8c\nInputFiles\x94\x93\x94)\x81\x94(\x8c7results/aggregated_counts/sars2_mito_aligned_by_run.csv\x94\x8cFresults/aggregated_counts/sars2_mito_aligned_by_metagenomic_sample.csv\x94\x8cAresults/aggregated_counts/mito_composition_by_metagenomic_run.csv\x94\x8cDresults/aggregated_counts/mito_composition_by_metagenomic_sample.csv\x94\x8c,results/crits_christoph_data/read_counts.csv\x94\x8c@results/crits_christoph_data/check_sha512_vs_crits_christoph.csv\x94e}\x94(\x8c\x06_names\x94}\x94(\x8c\x19sars2_mito_aligned_by_run\x94K\x00N\x86\x94\x8c(sars2_mito_aligned_by_metagenomic_sample\x94K\x01N\x86\x94\x8c#mito_composition_by_metagenomic_run\x94K\x02N\x86\x94\x8c&mito_composition_by_metagenomic_sample\x94K\x03N\x86\x94\x8c\x1bcrits_christoph_read_counts\x94K\x04N\x86\x94\x8c\x17ngdc_to_crits_christoph\x94K\x05N\x86\x94u\x8c\x12_allowed_overrides\x94]\x94(\x8c\x05index\x94\x8c\x04sort\x94eh!\x8c\tfunctools\x94\x8c\x07partial\x94\x93\x94h\x06\x8c\x19Namedlist._used_attribute\x94\x93\x94\x85\x94R\x94(h\')}\x94\x8c\x05_name\x94h!sNt\x94bh"h%h\'\x85\x94R\x94(h\')}\x94h+h"sNt\x94b\x8c\x19sars2_mito_aligned_by_run\x94h\n\x8c(sars2_mito_aligned_by_metagenomic_sample\x94h\x0b\x8c#mito_composition_by_metagenomic_run\x94h\x0c\x8c&mito_composition_by_metagenomic_sample\x94h\rh\x1bh\x0eh\x1dh\x0fub\x8c\x06output\x94h\x06\x8c\x0bOutputFiles\x94\x93\x94)\x81\x94\x8c\rresults/plots\x94a}\x94(h\x11}\x94\x8c\x08plotsdir\x94K\x00N\x86\x94sh\x1f]\x94(h!h"eh!h%h\'\x85\x94R\x94(h\')}\x94h+h!sNt\x94bh"h%h\'\x85\x94R\x94(h\')}\x94h+h"sNt\x94bh<h9ub\x8c\x06params\x94h\x06\x8c\x06Params\x94\x93\x94)\x81\x94]\x94\x8c\x0bcollections\x94\x8c\x0bOrderedDict\x94\x93\x94)R\x94\x8c\rAmur hedgehog\x94\x8c\x13Erinaceus amurensis\x94saa}\x94(h\x11}\x94\x8c-mitochondrial_genomes_crits_christoph_missing\x94K\x00N\x86\x94sh\x1f]\x94(h!h"eh!h%h\'\x85\x94R\x94(h\')}\x94h+h!sNt\x94bh"h%h\'\x85\x94R\x94(h\')}\x94h+h"sNt\x94bhThKub\x8c\twildcards\x94h\x06\x8c\tWildcards\x94\x93\x94)\x81\x94}\x94(h\x11}\x94h\x1f]\x94(h!h"eh!h%h\'\x85\x94R\x94(h\')}\x94h+h!sNt\x94bh"h%h\'\x85\x94R\x94(h\')}\x94h+h"sNt\x94bub\x8c\x07threads\x94K\x01\x8c\tresources\x94h\x06\x8c\tResources\x94\x93\x94)\x81\x94(K\x01K\x01\x8c\x04/tmp\x94e}\x94(h\x11}\x94(\x8c\x06_cores\x94K\x00N\x86\x94\x8c\x06_nodes\x94K\x01N\x86\x94\x8c\x06tmpdir\x94K\x02N\x86\x94uh\x1f]\x94(h!h"eh!h%h\'\x85\x94R\x94(h\')}\x94h+h!sNt\x94bh"h%h\'\x85\x94R\x94(h\')}\x94h+h"sNt\x94bhvK\x01hxK\x01hzhsub\x8c\x03log\x94h\x06\x8c\x03Log\x94\x93\x94)\x81\x94\x8c\x18results/make_plots.ipynb\x94a}\x94(h\x11}\x94\x8c\x08notebook\x94K\x00N\x86\x94sh\x1f]\x94(h!h"eh!h%h\'\x85\x94R\x94(h\')}\x94h+h!sNt\x94bh"h%h\'\x85\x94R\x94(h\')}\x94h+h"sNt\x94bh\x8ch\x89ub\x8c\x06config\x94}\x94(\x8c\x14crits_christoph_data\x94}\x94(\x8c\tfile_info\x94\x8c\x95https://raw.githubusercontent.com/sars-cov-2-origins/huanan-environmental/bd2afdb19392a0efb872c09eb94045de9d265a6e/tables/TableS1_FileInformation.csv\x94\x8c\x0csample_names\x94\x8c\x91https://raw.githubusercontent.com/sars-cov-2-origins/huanan-environmental/bd2afdb19392a0efb872c09eb94045de9d265a6e/tables/TableS2_SampleNames.csv\x94\x8c\x0bread_counts\x94\x8c\x96https://raw.githubusercontent.com/sars-cov-2-origins/huanan-environmental/bd2afdb19392a0efb872c09eb94045de9d265a6e/tables/TableS3_MappedReadCounts.csv\x94u\x8c\x15mitochondrial_genomes\x94\x8cYhttps://ftp.ncbi.nlm.nih.gov/refseq/release/mitochondrion/mitochondrion.1.genomic.gbff.gz\x94\x8c\x1amitochondrial_taxa_to_keep\x94}\x94(\x8c\x06phylum\x94]\x94\x8c\x08Chordata\x94a\x8c\tsubphylum\x94]\x94u\x8c\x1dmitochondrial_genomes_to_keep\x94}\x94(\x8c\x03pig\x94\x8c\x15Sus scrofa domesticus\x94\x8c\tbrown rat\x94\x8c\x1fRattus norvegicus strain Wistar\x94\x8c\x06rabbit\x94\x8c\x15Oryctolagus cuniculus\x94\x8c\x03dog\x94\x8c\x16Canis lupus familiaris\x94\x8c\x11European hedgehog\x94\x8c\x13Erinaceus europaeus\x94\x8c\x13long-eared hedgehog\x94\x8c\x13Hemiechinus auritus\x94\x8c\x03cow\x94\x8c\nBos taurus\x94\x8c\x05human\x94\x8c\x0cHomo sapiens\x94\x8c\x0braccoon dog\x94\x8c\x18Nyctereutes procyonoides\x94\x8c\x10Himalayan marmot\x94\x8c\x12Marmota himalayana\x94\x8c\x0fSiberian weasel\x94\x8c\x10Mustela sibirica\x94\x8c\nhog badger\x94\x8c\x11Arctonyx collaris\x94\x8c\x10hoary bamboo rat\x94\x8c\x12Rhizomys pruinosus\x94\x8c\nfennec fox\x94\x8c\x0cVulpes zerda\x94\x8c\x05sheep\x94\x8c\x0fOvis orientalis\x94\x8c\x08gray fox\x94\x8c\x18Urocyon cinereoargenteus\x94\x8c\x04goat\x94\x8c\x0eCapra aegagrus\x94\x8c\x08sand cat\x94\x8c\x0fFelis margarita\x94\x8c\x11masked palm civet\x94\x8c\x0ePaguma larvata\x94\x8c\x11Mayalan field rat\x94\x8c\x11Rattus tiomanicus\x94\x8c\rbat-eared fox\x94\x8c\x11Otocyon megalotis\x94\x8c\tleaf deer\x94\x8c\x14Muntiacus putaoensis\x94\x8c\x11Malayan porcupine\x94\x8c\x11Hystrix brachyura\x94\x8c\x03cat\x94\x8c\x0bFelis catus\x94\x8c\x07chicken\x94\x8c\x17Gallus gallus spadiceus\x94\x8c\x04duck\x94\x8c\x13Anas poecilorhyncha\x94\x8c\tcrocodile\x94\x8c\x15Osteolaemus tetraspis\x94\x8c\x13Japanese salamander\x94\x8c\x11Andrias japonicus\x94\x8c\trat snake\x94\x8c\x0cElaphe dione\x94\x8c\x0esnakehead fish\x94\x8c\x0fChanna maculata\x94\x8c\x0ebrown bush hen\x94\x8c\x10Amaurornis akool\x94\x8c\rcollared dove\x94\x8c\x15Streptopelia decaocto\x94\x8c\x0fgolden pheasant\x94\x8c\x13Chrysolophus pictus\x94u\x8c-mitochondrial_genomes_crits_christoph_missing\x94]\x94hOa\x8c"mitochondrial_genome_min_mash_dist\x94G?\xb9\x99\x99\x99\x99\x99\x9a\x8c\tsars2_ref\x94\x8cGhttp://hgdownload.soe.ucsc.edu/goldenPath/wuhCor1/bigZips/wuhCor1.fa.gz\x94\x8c\x0csars2_ref_id\x94\x8c\x0bNC_045512v2\x94\x8c\x18metagenomic_descriptions\x94]\x94\x8cORNA sequencing of total nucleic acids from environmental swabs for metagenomics\x94a\x8c\x18mito_composition_filters\x94}\x94(\x8c\x19min_alignments_run_filter\x94M\xd0\x07\x8c!min_percent_aligned_genome_filter\x94K\nuu\x8c\x04rule\x94\x8c\nmake_plots\x94\x8c\x0fbench_iteration\x94N\x8c\tscriptdir\x94\x8cT/fh/fast/bloom_j/computational_notebooks/jbloom/2023/Huanan_market_samples/notebooks\x94ub.'); from snakemake.logging import logger; logger.printshellcmds = False; import os; os.chdir(r'/fh/fast/bloom_j/computational_notebooks/jbloom/2023/Huanan_market_samples');
######## snakemake preamble end #########


Import Python modules:

In [72]:
import itertools
import os

import altair as alt

import pandas as pd

Get data and variable from `snakemake`:

In [5]:
sars2_mito_aligned_by_run = pd.read_csv(snakemake.input.sars2_mito_aligned_by_run)

sars2_mito_aligned_by_metagenomic_sample = pd.read_csv(snakemake.input.sars2_mito_aligned_by_metagenomic_sample)

mito_composition_by_metagenomic_run = pd.read_csv(snakemake.input.mito_composition_by_metagenomic_run)

mito_composition_by_metagenomic_sample = pd.read_csv(snakemake.input.mito_composition_by_metagenomic_sample)

crits_christoph_read_counts = pd.read_csv(snakemake.input.crits_christoph_read_counts)

ngdc_to_crits_christoph = pd.read_csv(snakemake.input.ngdc_to_crits_christoph)

plotsdir = snakemake.output.plotsdir
mitochondrial_genomes_crits_christoph_missing = snakemake.params.mitochondrial_genomes_crits_christoph_missing

## Compare mitochondrial DNA composition to Crits-Christoph et al
First get Crits-Christoph read counts in tidy format, assigning NGDC run accessions and summing counts for runs:

In [47]:
crits_christoph_read_counts_tidy = (
    crits_christoph_read_counts
    .drop(columns=["Location", "Sample_name"])
    .merge(
        ngdc_to_crits_christoph
        .assign(Filename=lambda x: x["fastq Crits-Christoph"].map(lambda s: s.split(".")[0]))
        .rename(columns={"Run accession NGDC": "Run accession"})
        [["Filename", "Run accession"]]
        .drop_duplicates(),
        validate="one_to_one",
    )
    .merge(
        sars2_mito_aligned_by_run[["Run accession", "sample"]].drop_duplicates(),
        validate="many_to_one",
    )
    .melt(
        id_vars=["sample", "Run accession", "Filename"],
        var_name="species",
        value_name="alignment count Crits-Christoph",
    )
    .groupby(["sample", "Run accession", "species"], as_index=False)
    .aggregate({"alignment count Crits-Christoph": "sum"})
)

crits_christoph_read_counts_tidy

,sample,Run accession,species,alignment count Crits-Christoph
0,1-29-8,CRR710770,Arctonyx collaris,0
1,1-29-8,CRR710770,Bos taurus,0
2,1-29-8,CRR710770,Canis lupus,0
3,1-29-8,CRR710770,Capra aegagrus,0
4,1-29-8,CRR710770,Erinaceus amurensis,0
...,...,...,...,...
2923,wws-1#-0,CRR711161,Rattus tiomanicus,0
2924,wws-1#-0,CRR711161,Rhizomys pruinosus,0
2925,wws-1#-0,CRR711161,Sus scrofa,32
2926,wws-1#-0,CRR711161,Urocyon cinereoargenteus,0


Now merge current read counts with those from Crits-Christoph for all species:

In [90]:
# species specifically to drop
species_to_drop = {
    s
    for d in mitochondrial_genomes_crits_christoph_missing
    for _, s in d.items()
}

species_to_compare = [
    s for s in crits_christoph_read_counts_tidy["species"].unique()
    if s not in species_to_drop
]

# get current study data in form to compare, harmozing species names
mito_composition_to_compare = (
    mito_composition_by_metagenomic_run
    .assign(
        species=lambda x: x["species"].map(lambda s: " ".join(s.split()[: 2]))
    )
    [["Run accession", "species", "common_name", "alignment_count_type", "alignment_count"]]
)

# expand mito composition by run to include all Crits-Christoph species
assert set(crits_christoph_species_to_compare).issubset(
    mito_composition_to_compare["species"]
)

mito_composition_to_compare = (
    mito_composition_to_compare
    .drop(columns="common_name")
    .merge(
        pd.DataFrame(
            itertools.product(
                mito_composition_to_compare["Run accession"].unique(),
                species_to_compare,
                mito_composition_to_compare["alignment_count_type"].unique(),
            ),
            columns=["Run accession", "species", "alignment_count_type"],
        ),
        how="outer",
    )
    .assign(alignment_count=lambda x: x["alignment_count"].fillna(0).astype(int))
    .merge(
        mito_composition_to_compare[["species", "common_name"]].drop_duplicates(),
        validate="many_to_one",
        how="left",
    )
    .rename(columns={"alignment_count": "alignment count current study"})
)

crits_christoph_vs_current = (
    crits_christoph_read_counts_tidy
    .query("species not in @species_to_drop")
    .merge(
        mito_composition_to_compare,
        validate="one_to_many",
        on=["Run accession", "species"],
    )
)

crits_christoph_vs_current

,sample,Run accession,species,alignment count Crits-Christoph,alignment_count_type,alignment count current study,common_name
0,8-25-M1,CRR710772,Arctonyx collaris,0,n_primary,0,hog badger
1,8-25-M1,CRR710772,Arctonyx collaris,0,n_primary_and_unique_best,0,hog badger
2,8-25-M1,CRR710772,Arctonyx collaris,0,n_primary_or_equally_good_secondary,0,hog badger
3,8-25-M1,CRR710772,Bos taurus,0,n_primary,0,cow
4,8-25-M1,CRR710772,Bos taurus,0,n_primary_and_unique_best,0,cow
...,...,...,...,...,...,...,...
6895,wws-1#-0,CRR711161,Urocyon cinereoargenteus,0,n_primary_and_unique_best,2,gray fox
6896,wws-1#-0,CRR711161,Urocyon cinereoargenteus,0,n_primary_or_equally_good_secondary,232,gray fox
6897,wws-1#-0,CRR711161,Vulpes zerda,0,n_primary,1,fennec fox
6898,wws-1#-0,CRR711161,Vulpes zerda,0,n_primary_and_unique_best,1,fennec fox
